In [11]:
import pandas as pd

# API calls
from ibm_watson import PersonalityInsightsV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException


In [12]:
data = pd.read_csv('data/player_by_season_nlu_scores.csv')

In [13]:
example = data['cleaned_text'].iloc[0]
data.columns

Index(['twitter_handle', 'season', 'cleaned_text', 'doc_labels',
       'doc_labels_weights', 'emot_anger', 'emot_disgust', 'emot_fear',
       'emot_joy', 'emot_sadness', 'sent_class', 'sent_score'],
      dtype='object')

In [14]:
not_my_key = 'Tj4Y-bfKv_6IIf7NhubDpm6SAG8M2W22tQfbO2YVCBu1'
#z_key = 'QUSJ0cfugdJiYk5swh-XCK1t3RX9MgCPiWpIuchlX7eZ'
authenticator = IAMAuthenticator(not_my_key)

In [15]:
personality_insights = PersonalityInsightsV3(
    version='2017-10-13',
    authenticator=authenticator
)

In [16]:
not_my_url = 'https://api.us-south.personality-insights.watson.cloud.ibm.com/instances/7a03f7cd-ce72-4dc5-b755-c81a6b5a4aba'
personality_insights.set_service_url(not_my_url)

In [17]:
def get_personality_score(text, pi, handle, season):
    # call the API
    try:
        profile = pi.profile(text, 'application/json')
        p = profile.get_result()

        # personality columns
        needs_cols = [p['needs'][i]['trait_id'] for i in range(len(p['needs']))]
        personality_cols = []
        for i in range(len(p['personality'])):
            for j in range(len(p['personality'][i]['children'])):
                personality_cols.append(p['personality'][i]['children'][j]['trait_id'])
        value_cols = [p['values'][i]['trait_id'] for i in range(len(p['values']))]

        # personality values
        player_needs = [p['needs'][i]['percentile'] for i in range(len(p['needs']))]
        player_pers = []
        for i in range(len(p['personality'])):
            for j in range(len(p['personality'][i]['children'])):
                player_pers.append(p['personality'][i]['children'][j]['percentile'])
        player_values = [p['values'][i]['percentile'] for i in range(len(p['values']))]

        # extend the values
        needs_cols.extend(personality_cols)
        needs_cols.extend(value_cols)
        player_needs.extend(player_pers)
        player_needs.extend(player_values)

        # create a DF row
        year_personality = pd.DataFrame(columns=needs_cols)
        year_personality.loc[0] = player_needs
        year_personality['twitter_handle'] = [handle]
        year_personality['season'] = [season]

        # return the row
        return year_personality, True
    except:
        return pd.DataFrame(), False

In [18]:
def get_all_personality_scores(text_df, pi):
    cols = ['need_challenge','need_closeness','need_curiosity',
             'need_excitement','need_harmony',
             'need_ideal','need_liberty',
             'need_love','need_practicality',
             'need_self_expression',
             'need_stability',
             'need_structure',
             'facet_adventurousness',
             'facet_artistic_interests',
             'facet_emotionality',
             'facet_imagination',
             'facet_intellect',
             'facet_liberalism',
             'facet_achievement_striving',
             'facet_cautiousness',
             'facet_dutifulness',
             'facet_orderliness',
             'facet_self_discipline',
             'facet_self_efficacy',
             'facet_activity_level',
             'facet_assertiveness',
             'facet_cheerfulness',
             'facet_excitement_seeking',
             'facet_friendliness',
             'facet_gregariousness',
             'facet_altruism',
             'facet_cooperation',
             'facet_modesty',
             'facet_morality',
             'facet_sympathy',
             'facet_trust',
             'facet_anger',
             'facet_anxiety',
             'facet_depression',
             'facet_immoderation',
             'facet_self_consciousness',
             'facet_vulnerability',
             'value_conservation',
             'value_openness_to_change',
             'value_hedonism',
             'value_self_enhancement',
             'value_self_transcendence',
             'twitter_handle']
    personalities = pd.DataFrame(columns=cols)
    missing = pd.DataFrame(columns=['twitter_handle', 'season'])
    num_missing = 0
    for i in range(len(text_df)):
        hndl = text_df['twitter_handle'].iloc[i]
        txt = text_df['cleaned_text'].iloc[i]
        yr = text_df['season'].iloc[i]
        p_row, flag = get_personality_score(txt, pi, hndl, yr)
        if flag:
            personalities = pd.concat([personalities, p_row])
        else:
            missing[num_missing] = [hndl, yr]
            num_missing += 1
    
    return personalities, missing, num_missing

In [19]:
test = data.sample(2)

In [20]:
# Testing!

test_p, test_missing, test_num_missing = get_all_personality_scores(test, personality_insights)


ERROR:root:Forbidden
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403 , X-global-transaction-id: bf030b50-f75a-4aea-b36b-f7fd49ec61c5
ERROR:root:Forbidden
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403 , X-global-transaction-id: 2f72b4e2-1828-4f04-9a10-ddd41725c8fc


In [31]:
test_m = pd.merge(test, test_p, on="twitter_handle")

In [33]:
test_m

,twitter_handle,season,cleaned_text,doc_labels,doc_labels_weights,emot_anger,emot_disgust,emot_fear,emot_joy,emot_sadness,...,facet_anxiety,facet_depression,facet_immoderation,facet_self_consciousness,facet_vulnerability,value_conservation,value_openness_to_change,value_hedonism,value_self_enhancement,value_self_transcendence
0,GraysonJAllen,2012-2013,s/o to my bro !! million views on this crazy d...,/religion and spirituality/christianity /sport...,0.751638 0.748734 0.713964 0.691112 0.667692 0...,0.084902,0.065864,0.107672,0.654156,0.157753,...,0.08228,0.108009,0.06928,0.152954,0.121102,0.991351,0.729002,0.166152,0.210352,0.735793


In [34]:
# RUNNING FOR REAL
p_scores, missing, num_missing = get_all_personality_scores(data, personality_insights)

ERROR:root:The number of words 51 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 51 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: f736f95b-2766-4173-a170-67a8fd923da3
ERROR:root:The number of words 59 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 59 is less than the minimum number of words required for analysis: 100, Code: 400 , X-gl

ERROR:root:The number of words 78 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 78 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: c29018b8-cbc8-4ee2-befa-82933426ad83
ERROR:root:The number of words 7 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 7 is less than the minimum number of words required for analysis: 100, Code: 400 , X-glob

ERROR:root:The number of words 46 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 46 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: a44364cb-6260-4ccd-91c0-fad5e3ed9059
ERROR:root:The number of words 81 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 81 is less than the minimum number of words required for analysis: 100, Code: 400 , X-gl

ERROR:root:The number of words 8 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 8 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: c6327ae2-3ec1-4b4f-89d8-fbd3f64345a6
ERROR:root:The number of words 15 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 15 is less than the minimum number of words required for analysis: 100, Code: 400 , X-glob

ERROR:root:The number of words 9 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 9 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: eea8514b-29b7-4f00-beb6-b3523a3daf27
ERROR:root:The number of words 26 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 26 is less than the minimum number of words required for analysis: 100, Code: 400 , X-glob

...

ERROR:root:The number of words 68 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 68 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: cb298c1d-b7f1-491b-9a04-b63bb2e75079
ERROR:root:The number of words 68 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 68 is less than the minimum number of words required for analysis: 100, Code: 400 , X-gl

ERROR:root:The number of words 20 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 20 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: a3bdbd14-b9fc-4092-a5bd-36f0122e5d0b
ERROR:root:The number of words 26 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 26 is less than the minimum number of words required for analysis: 100, Code: 400 , X-gl

ERROR:root:The number of words 80 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 80 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: f9c5de66-8ed2-4e0b-91cf-cb54c1905750
ERROR:root:The number of words 9 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 9 is less than the minimum number of words required for analysis: 100, Code: 400 , X-glob

ERROR:root:The number of words 89 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 89 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: 5e545d89-c8e9-4ad9-b0c1-b3ff65f39a08
ERROR:root:The number of words 19 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 19 is less than the minimum number of words required for analysis: 100, Code: 400 , X-gl

ERROR:root:The number of words 34 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 34 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: 74ccc5cc-7ea5-4c7f-9cc0-6648bb054d01
ERROR:root:The number of words 40 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/home/mparkerseegmiller/.local/lib/python3.5/site-packages/ibm_cloud_sdk_core/base_service.py", line 246, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 40 is less than the minimum number of words required for analysis: 100, Code: 400 , X-gl

In [79]:
# let's find the missing values, we should have several
remove_rows = []
for i in range(len(data)):
    hndl = data['twitter_handle'].iloc[i]
    txt = data['cleaned_text'].iloc[i]
    yr = data['season'].iloc[i]
    l = len(txt.split())
    if l < 106:
        remove_rows.append(i)
len(remove_rows)

247

In [80]:
data.iloc[3]

twitter_handle                                                  alexlen
season                                                        2019-2020
cleaned_text          : theres no doubt in my mind, though. im the s...
doc_labels            /art and entertainment/visual art and design/d...
doc_labels_weights    0.704188 0.584926 0.571904 0.566116 0.565169 0...
emot_anger                                                      0.02107
emot_disgust                                                   0.253871
emot_fear                                                      0.139693
emot_joy                                                       0.469199
emot_sadness                                                   0.227202
sent_class                                                     positive
sent_score                                                     0.526574
Name: 3, dtype: object

In [92]:
data_big_enough = data.drop(remove_rows, axis=0)
# this worked, I believe

In [111]:
trial = pd.concat([data_big_enough, p_scores], axis = 1)

In [112]:
len(p_scores) == len(trial)

True

In [113]:
print("We have a total of "+str(len(data) - len(p_scores))+" years where there are less than 100 words...")

We have a total of 247 years where there are less than 100 words...


In [115]:
trial.sample(30)

,index,twitter_handle,season,cleaned_text,doc_labels,doc_labels_weights,emot_anger,emot_disgust,emot_fear,emot_joy,...,facet_depression,facet_immoderation,facet_self_consciousness,facet_vulnerability,value_conservation,value_openness_to_change,value_hedonism,value_self_enhancement,value_self_transcendence,twitter_handle
2116,2319,KevKnox,2014-2015,: congrats - from crusader nation! happy birth...,/sports/basketball /sports/boxing /sports/go k...,0.747014 0.681377 0.65271 0.64451 0.604149 0.5...,0.054373,0.089358,0.052857,0.685575,...,0.014270,0.214565,0.007345,0.077609,0.986330,0.219456,0.648820,0.558400,0.283210,KevKnox
202,217,TDORSEY_1,2014-2015,": huge th qt by (10pts)! st of group d, serbia...",/sports/basketball /sports/go kart /art and en...,0.86725 0.731547 0.71974 0.656952 0.640979 0.6...,0.057332,0.126856,0.067682,0.644240,...,0.144318,0.356269,0.368439,0.227472,0.878379,0.192657,0.632151,0.393361,0.092769,TDORSEY_1
1076,1185,joshhart,2020-2021,sounds terrible lol lol thats what i watched.....,/art and entertainment/shows and events /sport...,0.768944 0.751043 0.639265 0.614739 0.610979 0...,0.123725,0.064263,0.110019,0.595232,...,0.183940,0.742450,0.185391,0.165159,0.750847,0.604261,0.702903,0.755530,0.443661,joshhart
1402,1529,Dloading,2020-2021,: when dangelo russell called game at dyckman ...,/sports/basketball /sports/go kart /art and en...,0.764448 0.720948 0.710385 0.651803 0.632875 0...,0.524770,0.161699,0.083728,0.630309,...,0.095344,0.408976,0.313945,0.094456,0.977979,0.541518,0.793624,0.757423,0.407509,Dloading
539,578,youngwhiteside,2011-2012,ya man good hot dog n sorry abt that girl she ...,/art and entertainment/shows and events /sport...,0.740844 0.720218 0.69835 0.678265 0.643214 0....,0.111668,0.473545,0.085492,0.626268,...,0.146749,0.454360,0.210596,0.190256,0.965791,0.673774,0.753149,0.779780,0.668782,youngwhiteside
679,732,DamienINGLIS,2018-2019,one person followed me and one person unfollow...,/technology and computing/operating systems /t...,0.65723 0.63692 0.617419 0.593027 0.586682 0.5...,0.000000,0.000000,0.000107,0.024245,...,0.826003,0.701485,0.680126,0.788864,0.138793,0.341008,0.538823,0.385777,0.971652,DamienINGLIS
1798,1963,DevinBooker31,2016-2017,i love the offseason and all but i'm ready to ...,/sports/go kart /art and entertainment/shows a...,0.760772 0.729225 0.674307 0.659387 0.606819 0...,0.096067,0.106655,0.115393,0.659030,...,0.039131,0.528525,0.205197,0.160032,0.975103,0.451047,0.827020,0.585265,0.560591,DevinBooker31
1128,1240,vernon_macklin,2020-2021,nights be the toughest times....r.i.p pops i l...,/sports/go kart /art and entertainment/shows a...,0.707665 0.677691 0.63254 0.610233 0.598051 0....,0.073108,0.061904,0.103516,0.658812,...,0.025977,0.162582,0.085445,0.101038,0.926710,0.896794,0.769264,0.678460,0.808312,vernon_macklin
520,558,KlayThompson,2013-2014,dang that hurts clint dempsey!!!! i'm elated t...,/art and entertainment/shows and events /sport...,0.729674 0.690752 0.622215 0.620416 0.6161 0.6...,0.059037,0.064607,0.067720,0.730674,...,0.079767,0.355872,0.109323,0.105232,0.982735,0.492401,0.699994,0.729437,0.435136,KlayThompson
537,576,youngwhiteside,2012-2013,ill be back in july the most memorable moments...,/art and entertainment/shows and events /sport...,0.779101 0.748198 0.73543 0.708931 0.691199 0....,0.453116,0.101520,0.109166,0.655081,...,0.065206,0.362374,0.207755,0.150715,0.974743,0.716640,0.735604,0.771076,0.650411,youngwhiteside


In [116]:
len(p_scores) == len(data_big_enough)

True

In [117]:
#trial.to_csv('data/all_scores_by_season_edited.csv',index=False)

In [110]:
p_scores = p_scores.reset_index()
data_big_enough = data_big_enough.reset_index()

In [87]:
p_scores_clean = p_scores.copy()

In [47]:
#all_scores = pd.merge(data, p_scores, on="twitter_handle", how='right')

In [38]:
#all_scores.to_csv('data/all_scores_by_season.csv',index=False)

In [48]:
len(all_scores)

20060

In [ ]:
# personality columns
needs_cols = [p['needs'][i]['trait_id'] for i in range(len(p['needs']))]
personality_cols = []
for i in range(len(p['personality'])):
    for j in range(len(p['personality'][i]['children'])):
        personality_cols.append(p['personality'][i]['children'][j]['trait_id'])
value_cols = [p['values'][i]['trait_id'] for i in range(len(p['values']))]

# personality values
player_needs = [p['needs'][i]['percentile'] for i in range(len(p['needs']))]
player_pers = []
for i in range(len(p['personality'])):
    for j in range(len(p['personality'][i]['children'])):
        player_pers.append(p['personality'][i]['children'][j]['percentile'])
player_values = [p['values'][i]['percentile'] for i in range(len(p['values']))]



In [ ]:
needs_cols.extend(personality_cols)
needs_cols.extend(value_cols)
player_needs.extend(player_pers)
player_needs.extend(player_values)

In [ ]:
year_personality = pd.DataFrame(columns=needs_cols)
year_personality.loc[0] = player_needs

In [ ]:
year_personality

In [ ]:
needs_cols

In [ ]:
needs_cols = [p['needs'][i]['trait_id'] for i in range(len(p['needs']))]
print(needs_cols)
#for i in range(len(p['needs'])):
#    print(p['needs'][i]['percentile'])

In [ ]:
personality_cols = []
for i in range(len(p['personality'])):
    for j in range(len(p['personality'][i]['children'])):
        personality_cols.append(p['personality'][i]['children'][j]['trait_id'])
print(personality_cols)

In [ ]:
value_cols = [p['values'][i]['trait_id'] for i in range(len(p['values']))]
value_cols

In [ ]:
len(needs_cols) + len(personality_cols) + len(value_cols)

In [ ]:
for col in needs_cols:
    print(col+',')
for col in personality_cols:
    print(col+',')
for col in value_cols:
    print(col+',')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
needs_cols = [p['needs'][i]['trait_id'] for i in range(len(p['needs']))]
isaac_needs = [p['needs'][i]['percentile'] for i in range(len(p['needs']))]

In [ ]:
plt.barh(needs_cols,isaac_needs)
plt.xticks(rotation='vertical')
plt.suptitle('')
plt.title('Mrgn is Needy')
plt.xlabel('Percentile')
plt.ylabel('Need')
for index, value in enumerate(isaac_needs):
    plt.text(value, index, str(round(value,2)))
plt.show()

In [ ]:
personality_cols = []
isaac_pers = []
for i in range(len(p['personality'])):
    for j in range(len(p['personality'][i]['children'])):
        personality_cols.append(p['personality'][i]['children'][j]['trait_id'])
        isaac_pers.append(p['personality'][i]['children'][j]['percentile'])

In [ ]:
plt.barh(personality_cols[:10],isaac_pers[:10])
plt.yticks(rotation='horizontal')
plt.suptitle('')
plt.title('Mrgn Has a Personality')
plt.xlabel('Personality Measurement')
plt.ylabel('Percentile')
for index, value in enumerate(isaac_pers[:10]):
    plt.text(value, index, str(round(value,2)))
plt.show()

In [ ]:
plt.barh(personality_cols[10:18],isaac_pers[10:18])
plt.yticks(rotation='horizontal')
plt.suptitle('')
plt.title('Mrgn Has a Personality pt II')
plt.xlabel('Personality Measurement')
plt.ylabel('Percentile')
for index, value in enumerate(isaac_pers[10:18]):
    plt.text(value, index, str(round(value,2)))
plt.show()

In [ ]:
plt.barh(personality_cols[18:],isaac_pers[18:])
plt.yticks(rotation='horizontal')
plt.suptitle('')
plt.title('Mrgn Has a Personality pt III')
plt.xlabel('Personality Measurement')
plt.ylabel('Percentile')
for index, value in enumerate(isaac_pers[18:]):
    plt.text(value, index, str(round(value,2)))
plt.show()

In [ ]:
value_cols = [p['values'][i]['trait_id'] for i in range(len(p['values']))]
isaac_values = [p['values'][i]['percentile'] for i in range(len(p['values']))]

In [ ]:
plt.barh(value_cols,isaac_values)
plt.xticks(rotation='vertical')
plt.suptitle('')
plt.title('Mrgn Has Values??')
plt.xlabel('Value')
plt.ylabel('Percentile')
for index, value in enumerate(isaac_values):
    plt.text(value, index, str(round(value,2)))
plt.show()